Todo:
- Figure out user authentication
    - See if there's different syntax for working with user playlists
    - Probably don't go too deep w/ user authentication without planning a UI bc it's going to be tested by others
- Use artist and album to figure out label

In [31]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd
import numpy as np
from pprint import pprint
cid = '76c42909213b45a4a22596b35328705f'
secret = '9229ae6311b54925ad2f62ad4e32ccf6'
username = 'nshsunmalb8t3brxxz0ks3e47'
sp = spotipy.Spotify(client_credentials_manager=SpotifyClientCredentials(client_id=cid, client_secret=secret))

In [32]:
# Dictionary for mapping key signature integer value to pitch scale
int_to_key = {'-1': 'no key detected', '0': 'C', '1': 'C#', '2': 'D', '3': 'D#', '4': 'E',
              '5': 'E#', '6': 'F', '7': 'F#', '8': 'G', '9': 'G#', '10': 'A', '11': 'A#', '12': 'B'}

In [ ]:
# Transpose numeric key + boolean minor/major to camelot wheel
# '01': '8B', '00': '5A', '11': '3B', '10:'

In [33]:
# Example playlist: "wow im at the beach"
playlist_uri = 'spotify:playlist:5CF6KvWn85N6DoWufOjP5T'

In [56]:
offset = 0
jibjob = []

while True:
    response = sp.playlist_items(playlist_uri,
                                offset=offset,
                                fields='items.track.id,items.track.artists,items.track.name,items.track.album,total',
                                additional_types=['track'])
    
    if len(response['items']) == 0:
        # Combine inner lists and exit loop
        jibjob = [j for i in jibjob for j in i]
        break
    
    jibjob.append(response['items'])
    offset = offset + len(response['items'])
    print(offset, "/", response['total'])

100 / 209
200 / 209
209 / 209


In [57]:
df = pd.DataFrame(columns=['artist', 'track_name', 'id', 'album'], index=range(len(jibjob)))


In [58]:
for i in range(len(jibjob)):
    df['artist'].iloc[i] = jibjob[i]['track']['artists'][0]['name']
    df['track_name'].iloc[i] = jibjob[i]['track']['name']
    df['id'].iloc[i] = jibjob[i]['track']['id']
    df['album'].iloc[i] = jibjob[i]['track']['album']['name']


In [59]:
df

,artist,track_name,id,album
0,Tabu Ley Rochereau,Laiss Toi Aime',4MTiPNTHdAefzgJ7qgp3rY,Laiss Toi Aime
1,Negro Leo,Tudo Foi Feito pra Gente Lacrar,3k02yoh9Sjhjd6rKef1jyN,Desejo de Lacrar
2,Shad,Rose Garden,61kT5zIV1Z9sPbdAlKRExr,TSOL
3,Tabu Ley Rochereau,Marie Maria,2i7hxvOTfTjtsEJuWYnKg0,Laiss Toi Aime
4,Tabu Ley Rochereau,Mokrano,0gflenw36L3PVpVUqtRZJd,Laiss Toi Aime
...,...,...,...,...
204,Mdou Moctar,Tala Tannam - Debmaster Remix,4W3iduYwzo3hbpvOnN1Otw,Tala Tannam (Debmaster Remix)
205,The Cyrkle,The Visit (She Was Here),5dIcC2RTD2BOHTC1F0jrIN,Red Rubber Ball (A Collection)
206,Gilberto Gil,Era nova,6wPo638YVloLLGDPZRBE2F,Refavela
207,Paco de Lucía,Entre Dos Aguas - Instrumental,3Fgk17TfdjXWFroJy29pXH,Fuente Y Caudal


In [38]:
# Create DF for audio features while avoiding rate limit

features = []
for i in range(len(jibjob)):
    features.append(sp.audio_features(df['id'][i]))


In [39]:
features = [j for i in features for j in i]

In [40]:
df_features = pd.DataFrame(features)

In [41]:
df_features


,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,0.777,0.521,0,-11.669,1,0.0376,0.70100,0.585000,0.1780,0.852,132.733,audio_features,4MTiPNTHdAefzgJ7qgp3rY,spotify:track:4MTiPNTHdAefzgJ7qgp3rY,https://api.spotify.com/v1/tracks/4MTiPNTHdAef...,https://api.spotify.com/v1/audio-analysis/4MTi...,254520,4
1,0.573,0.408,8,-8.587,0,0.0373,0.92500,0.000147,0.0953,0.333,124.198,audio_features,3k02yoh9Sjhjd6rKef1jyN,spotify:track:3k02yoh9Sjhjd6rKef1jyN,https://api.spotify.com/v1/tracks/3k02yoh9Sjhj...,https://api.spotify.com/v1/audio-analysis/3k02...,235882,4
2,0.596,0.919,11,-4.451,1,0.3320,0.04930,0.000001,0.3050,0.835,183.331,audio_features,61kT5zIV1Z9sPbdAlKRExr,spotify:track:61kT5zIV1Z9sPbdAlKRExr,https://api.spotify.com/v1/tracks/61kT5zIV1Z9s...,https://api.spotify.com/v1/audio-analysis/61kT...,179200,4
3,0.550,0.341,10,-11.803,1,0.0414,0.73100,0.871000,0.1410,0.405,89.147,audio_features,2i7hxvOTfTjtsEJuWYnKg0,spotify:track:2i7hxvOTfTjtsEJuWYnKg0,https://api.spotify.com/v1/tracks/2i7hxvOTfTjt...,https://api.spotify.com/v1/audio-analysis/2i7h...,341040,4
4,0.696,0.582,9,-12.348,1,0.0410,0.55500,0.908000,0.1440,0.946,129.512,audio_features,0gflenw36L3PVpVUqtRZJd,spotify:track:0gflenw36L3PVpVUqtRZJd,https://api.spotify.com/v1/tracks/0gflenw36L3P...,https://api.spotify.com/v1/audio-analysis/0gfl...,326760,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
204,0.523,0.675,2,-8.344,1,0.0301,0.00617,0.017100,0.3720,0.196,84.038,audio_features,4W3iduYwzo3hbpvOnN1Otw,spotify:track:4W3iduYwzo3hbpvOnN1Otw,https://api.spotify.com/v1/tracks/4W3iduYwzo3h...,https://api.spotify.com/v1/audio-analysis/4W3i...,175714,4
205,0.696,0.133,7,-20.836,1,0.0387,0.71400,0.000010,0.1100,0.653,116.456,audio_features,5dIcC2RTD2BOHTC1F0jrIN,spotify:track:5dIcC2RTD2BOHTC1F0jrIN,https://api.spotify.com/v1/tracks/5dIcC2RTD2BO...,https://api.spotify.com/v1/audio-analysis/5dIc...,138133,4
206,0.528,0.446,7,-9.587,1,0.0360,0.77600,0.002530,0.0903,0.397,77.518,audio_features,6wPo638YVloLLGDPZRBE2F,spotify:track:6wPo638YVloLLGDPZRBE2F,https://api.spotify.com/v1/tracks/6wPo638YVloL...,https://api.spotify.com/v1/audio-analysis/6wPo...,238880,4
207,0.458,0.657,4,-12.020,0,0.0346,0.93300,0.878000,0.0842,0.914,102.450,audio_features,3Fgk17TfdjXWFroJy29pXH,spotify:track:3Fgk17TfdjXWFroJy29pXH,https://api.spotify.com/v1/tracks/3Fgk17TfdjXW...,https://api.spotify.com/v1/audio-analysis/3Fgk...,361867,4


In [49]:
merged = df.combine_first(df_features.drop(columns=['track_href', 'analysis_url', 'uri']))
merged

,acousticness,artist,danceability,duration_ms,energy,id,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,track_name,type,valence
0,0.70100,Tabu Ley Rochereau,0.777,254520,0.521,4MTiPNTHdAefzgJ7qgp3rY,0.585000,0,0.1780,-11.669,1,0.0376,132.733,4,Laiss Toi Aime',audio_features,0.852
1,0.92500,Negro Leo,0.573,235882,0.408,3k02yoh9Sjhjd6rKef1jyN,0.000147,8,0.0953,-8.587,0,0.0373,124.198,4,Tudo Foi Feito pra Gente Lacrar,audio_features,0.333
2,0.04930,Shad,0.596,179200,0.919,61kT5zIV1Z9sPbdAlKRExr,0.000001,11,0.3050,-4.451,1,0.3320,183.331,4,Rose Garden,audio_features,0.835
3,0.73100,Tabu Ley Rochereau,0.550,341040,0.341,2i7hxvOTfTjtsEJuWYnKg0,0.871000,10,0.1410,-11.803,1,0.0414,89.147,4,Marie Maria,audio_features,0.405
4,0.55500,Tabu Ley Rochereau,0.696,326760,0.582,0gflenw36L3PVpVUqtRZJd,0.908000,9,0.1440,-12.348,1,0.0410,129.512,4,Mokrano,audio_features,0.946
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
204,0.00617,Mdou Moctar,0.523,175714,0.675,4W3iduYwzo3hbpvOnN1Otw,0.017100,2,0.3720,-8.344,1,0.0301,84.038,4,Tala Tannam - Debmaster Remix,audio_features,0.196
205,0.71400,The Cyrkle,0.696,138133,0.133,5dIcC2RTD2BOHTC1F0jrIN,0.000010,7,0.1100,-20.836,1,0.0387,116.456,4,The Visit (She Was Here),audio_features,0.653
206,0.77600,Gilberto Gil,0.528,238880,0.446,6wPo638YVloLLGDPZRBE2F,0.002530,7,0.0903,-9.587,1,0.0360,77.518,4,Era nova,audio_features,0.397
207,0.93300,Paco de Lucía,0.458,361867,0.657,3Fgk17TfdjXWFroJy29pXH,0.878000,4,0.0842,-12.020,0,0.0346,102.450,4,Entre Dos Aguas - Instrumental,audio_features,0.914


In [50]:
for i in range(len(merged)):
    merged['key'].iloc[i] = int_to_key[str(merged['key'].iloc[i])]


/tmp/ipykernel_27795/1816759567.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged['key'].iloc[i] = int_to_key[str(merged['key'].iloc[i])]


In [51]:
merged

,acousticness,artist,danceability,duration_ms,energy,id,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,track_name,type,valence
0,0.70100,Tabu Ley Rochereau,0.777,254520,0.521,4MTiPNTHdAefzgJ7qgp3rY,0.585000,C,0.1780,-11.669,1,0.0376,132.733,4,Laiss Toi Aime',audio_features,0.852
1,0.92500,Negro Leo,0.573,235882,0.408,3k02yoh9Sjhjd6rKef1jyN,0.000147,G,0.0953,-8.587,0,0.0373,124.198,4,Tudo Foi Feito pra Gente Lacrar,audio_features,0.333
2,0.04930,Shad,0.596,179200,0.919,61kT5zIV1Z9sPbdAlKRExr,0.000001,A#,0.3050,-4.451,1,0.3320,183.331,4,Rose Garden,audio_features,0.835
3,0.73100,Tabu Ley Rochereau,0.550,341040,0.341,2i7hxvOTfTjtsEJuWYnKg0,0.871000,A,0.1410,-11.803,1,0.0414,89.147,4,Marie Maria,audio_features,0.405
4,0.55500,Tabu Ley Rochereau,0.696,326760,0.582,0gflenw36L3PVpVUqtRZJd,0.908000,G#,0.1440,-12.348,1,0.0410,129.512,4,Mokrano,audio_features,0.946
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
204,0.00617,Mdou Moctar,0.523,175714,0.675,4W3iduYwzo3hbpvOnN1Otw,0.017100,D,0.3720,-8.344,1,0.0301,84.038,4,Tala Tannam - Debmaster Remix,audio_features,0.196
205,0.71400,The Cyrkle,0.696,138133,0.133,5dIcC2RTD2BOHTC1F0jrIN,0.000010,F#,0.1100,-20.836,1,0.0387,116.456,4,The Visit (She Was Here),audio_features,0.653
206,0.77600,Gilberto Gil,0.528,238880,0.446,6wPo638YVloLLGDPZRBE2F,0.002530,F#,0.0903,-9.587,1,0.0360,77.518,4,Era nova,audio_features,0.397
207,0.93300,Paco de Lucía,0.458,361867,0.657,3Fgk17TfdjXWFroJy29pXH,0.878000,E,0.0842,-12.020,0,0.0346,102.450,4,Entre Dos Aguas - Instrumental,audio_features,0.914


In [52]:
merged['mode'] = np.where(merged['mode'] == 1, 'major', 'minor')
# merged_cond_test.loc[merged_cond_test['mode'] == 0, 'key'] = f"{merged_cond_test['key']} minor"

In [53]:
merged['key_signature'] = merged['key'] + ' ' + merged['mode']
merged['key_signature']

0       C major
1       G minor
2      A# major
3       A major
4      G# major
         ...   
204     D major
205    F# major
206    F# major
207     E minor
208    E# major
Name: key_signature, Length: 209, dtype: object

In [55]:
merged.query('time_signature != 4')

,acousticness,artist,danceability,duration_ms,energy,id,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,track_name,type,valence,key_signature
12,0.86800,The Melodians,0.667,158760,0.3970,50wl4ceK1FASG2rGkkp2fT,0.006150,C#,0.0786,-10.929,major,0.0453,167.825,3,It Comes and Goes,audio_features,0.8990,C# major
13,0.66800,Wax,0.527,241373,0.6570,0SyWTTxLNerKJchqgzw0JW,0.000000,A,0.4910,-6.056,minor,0.2670,78.348,1,Sunday in the City,audio_features,0.5630,A minor
41,0.92700,Rosinha De Valença,0.445,252413,0.3440,6p5QJBr5iE2xvHatGebTbR,0.738000,F#,0.0999,-11.311,major,0.0347,135.363,5,Asa Branca,audio_features,0.4290,F# major
57,0.85400,João Gilberto,0.458,397360,0.0352,2oLrUwf5PeLY2oB6K5O9jN,0.903000,G,0.1160,-27.066,major,0.0510,114.418,3,Undiú,audio_features,0.2000,G major
58,0.87600,João Gilberto,0.552,199227,0.0770,5oCezvp8Xm4k6ZgrnPenMc,0.871000,D,0.1110,-23.746,major,0.0300,102.142,3,Valsa (Como Sao Lindos os Youguis),audio_features,0.3920,D major
61,0.96800,Nara Leão,0.510,155867,0.1640,6JOUzBzOut78ryNFuCc39k,0.003710,E#,0.1060,-14.807,minor,0.0409,138.362,3,Corcovado,audio_features,0.2320,E# minor
62,0.92500,Nara Leão,0.543,122773,0.2430,6UOy5Ne0SGvbozHrLlLJ1J,0.000063,C,0.1210,-13.387,major,0.0517,107.743,5,Chega De Saudade,audio_features,0.3720,C major
65,0.34900,Tom Zé,0.667,198933,0.2060,1brWhQEoBoT9tMnNfcRsjr,0.000390,G#,0.1340,-14.884,minor,0.3250,148.450,3,A felicidade,audio_features,0.4080,G# minor
70,0.81200,Baden Powell,0.265,363267,0.2520,1FM6WoiD8RPTAzozx56wvi,0.132000,C#,0.0712,-18.445,minor,0.0354,96.737,3,Canto de Yemanjá,audio_features,0.1940,C# minor
88,0.13800,Jizzle,0.609,165619,0.7080,3pW9AlECzn3lmAEMAE9yce,0.000000,D,0.1270,-7.864,major,0.3150,67.121,5,BE MY LOVER,audio_features,0.6640,D major
